In [ ]:
import os
from os import path

# Third-party
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import h5py
import pandas as pd
from sqlalchemy import func
import tqdm

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import unimodal_P
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar,
                        StarResult, Status, JokerRun, initialize_db)
from twoface.plot import plot_two_panel

In [ ]:
plot_path = '../../paper/1-catalog/figures/'
table_path = '../../paper/1-catalog/tables/'

Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')

See: Sample K cuts.ipynb for setting `StarResult.high_K`

In [ ]:
high_K_stars = session.query(AllStar).join(StarResult).filter(StarResult.status_id>0).filter(StarResult.high_K).all()
low_K_stars = session.query(AllStar).join(StarResult).filter(StarResult.status_id>0).filter(~StarResult.high_K).all()
apogee_ids = np.array([x.apogee_id for x in high_K_stars])
len(high_K_stars), len(low_K_stars)

In [ ]:
n_martig = session.query(AllStar).join(StarResult).filter(StarResult.high_K).filter(AllStar.martig_filter).count()
print(n_martig)

In [ ]:
statuses = np.array([star.results[0].status_id for star in high_K_stars])
for i in range(4+1):
    count = session.query(StarResult).filter(StarResult.status_id == i).count()
    status = session.query(Status).filter(Status.id == i).one()
    print("Status: {1} [{0}]\n\t{2} in high K sample of {3} total\n"
          .format(i, status.message, np.sum(statuses == i), count))

In [ ]:
ranges = [np.arange(3, 9+1, 2), 
          np.arange(11, 14+1, 1)]

In [ ]:
np.random.seed(420)

stars = high_K_stars
for k, range_ in enumerate(ranges):
    gs = GridSpec(4, 3)
    fig = plt.figure(figsize=(12, 14))
    for j, i in enumerate(range_):
        ax1 = fig.add_subplot(gs[j, :2])
        ax2 = fig.add_subplot(gs[j, 2])
        
        if j == 0:
            ax1.set_title('Examples of high-$K$ stars', fontsize=20)
    
        sub_stars = session.query(AllStar).join(StarResult, AllVisitToAllStar, AllVisit)\
                           .filter(StarResult.status_id>0)\
                           .filter(StarResult.high_K)\
                           .filter(StarResult.status_id == 4)\
                           .filter(AllStar.logg > 1.5)\
                           .filter(AllStar.aspcapflag.op('&')(2**7) == 0)\
                           .group_by(AllStar.apstar_id)\
                           .having(func.count(AllVisit.id) == i).all()
        print(len(sub_stars))

        star = np.random.choice(sub_stars)
        data = star.apogeervdata()

        with h5py.File(samples_file, 'r') as f:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])

        fig = plot_two_panel(data, samples, axes=[ax1, ax2], tight=False,
                             plot_data_orbits_kw=dict(n_times=16384, 
                                                      highlight_P_extrema=False,
                                                      xlim_choice='data',
                                                      plot_kwargs=dict(alpha=0.2, 
                                                                       rasterized=True)))
        
        xlim = ax1.get_xlim()
        ylim = ax1.get_ylim()
        
        ax1.text(xlim[0] + (xlim[1]-xlim[0])/20,
                 ylim[1] - (ylim[1]-ylim[0])/20,
                 star.apogee_id, fontsize=18, va='top', ha='left')
        
        ax1.text(xlim[1] - (xlim[1]-xlim[0])/20,
                 ylim[1] - (ylim[1]-ylim[0])/20,
                 '$N = {0}$'.format(len(data)), 
                 fontsize=18, va='top', ha='right')
        
        ax1.set_xlabel('')
        ax2.set_xlabel('')
    
    ax1.set_xlabel('BMJD')
    ax2.set_xlabel('period, $P$ [day]')
    
    fig.tight_layout()
    fig.savefig(path.join(plot_path, 'highK-{0}.pdf'.format(k)), 
                dpi=150, rasterized=True)

In [ ]:
np.random.seed(420)

stars = low_K_stars
for k, range_ in enumerate(ranges):
    gs = GridSpec(4, 3)
    fig = plt.figure(figsize=(12, 14))
    for j, i in enumerate(range_):
        ax1 = fig.add_subplot(gs[j, :2])
        ax2 = fig.add_subplot(gs[j, 2])
        
        if j == 0:
            ax1.set_title('Examples of low-$K$ stars', fontsize=20)
    
        sub_stars = session.query(AllStar).join(StarResult, AllVisitToAllStar, AllVisit)\
                           .filter(StarResult.status_id>0)\
                           .filter(~StarResult.high_K)\
                           .filter(StarResult.status_id == 4)\
                           .filter(AllStar.logg > 1.5)\
                           .filter(AllStar.aspcapflag.op('&')(2**7) == 0)\
                           .group_by(AllStar.apstar_id)\
                           .having(func.count(AllVisit.id) == i).all()
        print(len(sub_stars))

        star = np.random.choice(sub_stars)
        data = star.apogeervdata()

        with h5py.File(samples_file, 'r') as f:
            samples = JokerSamples.from_hdf5(f[star.apogee_id])

        fig = plot_two_panel(data, samples, axes=[ax1, ax2], tight=False,
                             plot_data_orbits_kw=dict(n_times=16384, 
                                                      highlight_P_extrema=False,
                                                      xlim_choice='data',
                                                      plot_kwargs=dict(alpha=0.2, 
                                                                       rasterized=True)))
        
        xlim = ax1.get_xlim()
        ylim = ax1.get_ylim()
        
        ax1.text(xlim[0] + (xlim[1]-xlim[0])/20,
                 ylim[1] - (ylim[1]-ylim[0])/20,
                 star.apogee_id, fontsize=18, va='top', ha='left')
        
        ax1.text(xlim[1] - (xlim[1]-xlim[0])/20,
                 ylim[1] - (ylim[1]-ylim[0])/20,
                 '$N = {0}$'.format(len(data)), 
                 fontsize=18, va='top', ha='right')
        
        ax1.set_xlabel('')
        ax2.set_xlabel('')
    
    ax1.set_xlabel('BMJD')
    ax2.set_xlabel('period, $P$ [day]')
    
    fig.tight_layout()
    fig.savefig(path.join(plot_path, 'lowK-{0}.pdf'.format(k)), 
                dpi=150, rasterized=True)